In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import m_p, c, e

plt.rcParams['image.cmap'] = 'viridis'

In [2]:
# READ DATA
# =========
path = './cell_mon.h5'
# path = './cellmonitor2_0001_config_200.h5'
# path = './cellmonitor1_0002_config_400.h5'
# path = './cellmonitor2_0002_config_400.h5'
hf = h5py.File(path, 'r')
ha = hf['Cells']
n_azim_slices, n_rings, n_turns = ha['macroparticlenumber'].shape
radial_cut = hf.attrs.get('radual_cut')
beta_x = hf.attrs.get('beta_x')
beta_y = hf.attrs.get('beta_y')
beta_z = hf.attrs.get('beta_z')

In [4]:
plt.close('all')
# ANIMATION
# =========
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15,13), tight_layout=False,
                                    subplot_kw=dict(projection='polar'))
fig.subplots_adjust(hspace=0.3)

theta = np.linspace(-np.pi, np.pi, n_azim_slices)
r     = np.linspace(0, radial_cut, n_rings)
theta, r = np.meshgrid(theta, r)

var_Jx = ha['mean_x'][:,:,:]**2 #+ (beta_x*ha['mean_xp'][:,:,:])**2
var_Jy = ha['mean_y'][:,:,:]**2 #+ (beta_y*ha['mean_yp'][:,:,:])**2
vmax_x = np.max(10*np.log10(var_Jx.T + 1e-12))
vmax_y = np.max(10*np.log10(var_Jy.T + 1e-12))
x_levels = sorted(np.linspace(0.45, 1, 60) * vmax_x)
y_levels = sorted(np.linspace(0.45, 1, 60) * vmax_y)

plt.ion()
imovie = 0
for i in xrange(0, n_turns, 128):
    var_z = ha['mean_z'][:,:,i]
    var_dp = ha['mean_dp'][:,:,i]
    var_Jz = ha['mean_z'][:,:,i]**2 + (beta_z*ha['mean_dp'][:,:,i])**2
    var_Jx = ha['mean_x'][:,:,i]**2 #+ (beta_x*ha['mean_xp'][:,:,i])**2
    var_Jy = ha['mean_y'][:,:,i]**2 #+ (beta_y*ha['mean_yp'][:,:,i])**2

#     print np.max(10*np.log10(var_Jx.T + 1e-12))
#     print np.max(10*np.log10(var_Jy.T + 1e-12))

    var_z = np.ma.masked_equal(var_z, 0)
    var_dp = np.ma.masked_equal(var_dp, 0)
    var_Jz = np.ma.masked_equal(var_Jz, 0)
    var_Jx = np.ma.masked_equal(var_Jx, 0)
    var_Jy = np.ma.masked_equal(var_Jy, 0)


    ax1.contourf(theta, r, var_z.T, 60)
    ax2.contourf(theta, r, var_dp.T, 60)
    ax3.contourf(theta, r, 10*np.log10(var_Jx.T + 1e-10), levels=x_levels)
    ax4.contourf(theta, r, 10*np.log10(var_Jy.T + 1e-10), levels=y_levels)
#     ax3.contourf(theta, r, 10*np.log10(var_Jx.T + 1e-10), 60, vmin=vmax_x*0.15, vmax=vmax_x*0.5)
#     ax4.contourf(theta, r, 10*np.log10(var_Jy.T + 1e-10), 60, vmin=vmax_y*0.15, vmax=vmax_y*0.5)
    # ax3.contourf(theta, r, 10*(var_Jx.T + 1e-10)**3, 60)
    # ax4.contourf(theta, r, 10*(var_Jy.T + 1e-10)**3, 60)

    [ax.autoscale(tight=True) for ax in [ax1, ax2, ax3, ax4]]
    [ax.autoscale(tight=True) for ax in [ax1, ax2, ax3, ax4]]
    [ax.set_axis_bgcolor(plt.cm.viridis(0)) for ax in [ax1, ax2, ax3, ax4]]
    ax1.set_title(r'$z$', y=1.1, fontsize=20)
    ax2.set_title(r'$\delta$', y=1.1, fontsize=20)
    ax3.set_title(r'$\mu_x$', y=1.1, fontsize=20)
    ax4.set_title(r'$\mu_{xp}$', y=1.1, fontsize=20)

    plt.draw()
    plt.pause(0.1)
    [ax.cla() for ax in [ax1, ax2, ax3, ax4]]

/usr/lib/python2.7/site-packages/ipykernel/__main__.py:40: RuntimeWarning: divide by zero encountered in log10
/usr/lib/python2.7/site-packages/ipykernel/__main__.py:41: RuntimeWarning: divide by zero encountered in log10
/usr/lib64/python2.7/site-packages/matplotlib/backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


In [6]:
plt.close('all')